<a href="https://colab.research.google.com/github/Gitanjali-63/Image-segmentation-using-PSO/blob/main/PSO_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from natsort import natsorted 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import xgboost
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import os
%matplotlib inline

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
#DATA ANALYSIS
data=pd.read_csv('Brain Tumor.csv')
data

In [ ]:
# SCALING
scalable=['Mean', 'Variance', 'Standard Deviation', 'Entropy',
       'Skewness', 'Kurtosis', 'Contrast', 'Energy', 'ASM', 'Homogeneity',
       'Dissimilarity', 'Correlation', 'Coarseness']


data[scalable]=StandardScaler().fit_transform(data[scalable])
data

In [ ]:
#DATA CORRELATIONS
sns.swarmplot(x=y,y= data['Homogeneity'])
plt.title("Distribution of image Homogenity, by Class")

In [ ]:
class1=data['Class']== 1
class0=data['Class']== 0
_data=data.copy()
_data=data.drop('Image',axis=1,inplace=False)
sns.distplot(a= _data[class1]['Energy'], label="Tumor")
sns.distplot(a = _data[class0]['Energy'], label="No tumor" )

plt.title("Distribution of image Energy, by Class")
plt.legend()

In [ ]:
sns.distplot(a= _data[class1]['Entropy'], label="Tumor")
sns.distplot(a = _data[class0]['Entropy'], label="No tumor" )
plt.title("Distribution of image Entropy, by Class")
plt.legend()

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fig = plt.figure()  
folder='/content/Image3760.jpg'
imgs=natsorted(folder)

img=cv2.imread(folder,cv2.IMREAD_GRAYSCALE)

im = plt.imshow(img,  interpolation='none', aspect='auto',cmap ='gray', vmin=0, vmax=255)   
plt.title('No Tumor')

In [ ]:
fig = plt.figure()  
folder='/content/Image3.jpg'
imgs=natsorted(folder)

img=cv2.imread(folder,cv2.IMREAD_GRAYSCALE)

im = plt.imshow(img,  interpolation='none', aspect='auto',cmap ='gray', vmin=0, vmax=255)   
plt.title('Tumor')



In [ ]:
y=data.Class
y

In [ ]:
#Most impactful features
from collections import OrderedDict

model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 
model.fit(data.drop(['Image','Class'],axis=1,inplace=False),y)
OrderedDict(sorted(model.get_booster().get_fscore().items(),key=lambda t: t[1], reverse=True))

In [ ]:
#BRAIN TUMOR PREDICTION
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
yes_data = []

no_data = []

paths = []

labels = []

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Brain Tumor'):
    
    for filename in filenames:
        
        if '.jpg' in filename:
            
            paths.append(os.path.join(dirname, filename))

In [ ]:
encoder = OneHotEncoder()

encoder.fit([[0], [1]])

for path in paths:
    
    image = Image.open(path)
    
    image = image.resize((128,128))
    
    image= np.array(image)
    
    if image.shape == (128,128,3):
        
        yes_data.append(np.array(image))
        
        labels.append(encoder.transform([[0]]).toarray())
        
labels[0]

In [ ]:
more_paths = []

for dirname, _, filenames in os.walk('/content/drive/MyDrive/Brain Tumor/no'):
    
    for filename in filenames:
        
        if '.jpg' in filename:
            
            more_paths.append(os.path.join(dirname, filename))

In [ ]:
for path in more_paths:
    
    image = Image.open(path)
    
    image = image.resize((128,128))
    
    image = np.array(image)
    
    if image.shape == (128,128,3):
        
        yes_data.append(np.array(image))
        
        labels.append(encoder.transform([[1]]).toarray())

In [ ]:
yes_data = np.array(yes_data)

yes_data.shape

In [ ]:
labels = np.array(labels)

labels = labels.reshape(3762,2)

labels.shape

In [ ]:
np.set_printoptions(linewidth = 200)

plt.imshow(yes_data[0])

print(yes_data[0])

print(labels[0])

In [ ]:
#Training + Testing data
train_data, test_data, train_labels, test_labels = train_test_split(yes_data, labels, random_state = 3, shuffle = True)

In [ ]:
#Normalizing Data
train_data = train_data /255.0
test_data = test_data/255.0

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization


input = Input(shape = (128,128,3))

x = Conv2D(16, (3,3), activation = 'relu')(input)

x = BatchNormalization()(x)

x = MaxPooling2D(2,2)(x)

x = Conv2D(32, (3,3), activation = 'relu')(x)

x = MaxPooling2D(2,2)(x)



x = Flatten()(x)

x = BatchNormalization()(x)

x = Dense(124, activation = 'relu')(x)

x = Dropout(0.27)(x)

x = Dense(124, activation = 'relu')(x)

output = Dense(2, activation = 'softmax')(x)

model = Model(inputs = input, outputs = output)

model.summary()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import rmsprop_v2

In [ ]:
model.compile(loss = 'categorical_crossentropy',  metrics = ["accuracy"])

In [ ]:
# Making a Learning Rate Scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8*10**(epoch / 20))

In [ ]:
#Plotting results
# accuracy

plt.figure(figsize = (12,7))

plt.plot(history.history['accuracy'], label = 'train accuracy')
plt.plot(history.history['val_accuracy'], label = 'validation accuracy')
plt.title("train accuracy vs validation accuracy")
plt.legend()
plt.show()

In [ ]:
# loss

plt.figure(figsize = (12,7))

plt.plot(history.history['loss'], label = 'train loss')
plt.plot(history.history['val_loss'], label = 'validation loss')
plt.legend()
plt.title("train vs validation loss")
plt.show()

In [ ]:
#Predictions
def tumor(number):
    
    if number == 0:
        
        return "Not a tumor"
    
    elif nimber ==1:
        
        return "a tumor"

In [ ]:
#example image

img = Image.open("/content/Image3760.jpg")

x = np.array(img.resize((128,128)))

x = x.reshape(1, 128, 128, 3)

result = model.predict([x])

classification = np.where(result == np.amax(result))[1][0]

print(str(result[0][classification]*100) + '% Confidence This Is ' + tumor(classification))

plt.imshow(img)

In [ ]:
img2 = Image.open(r"/content/Image70.jpg")

y = np.array(img2.resize((128,128)))

y = y.reshape(1, 128, 128, 3)

result2 = model.predict([y])

ind = 0
classification2 = 0

import numpy as np

for result in result2:
    result = result.tolist()
    print(result)
    
    if result[0] > result[1]:
        classification = 1
        ind = 0
    else:
        classification = 0
        ind = 1

#classification2 = np.where(result2 == np.amax(result2))[1][0]

print(str(result2[0][ind]*100) + '% Confidence This Is ' + tumor(classification2))

plt.imshow(img2)

In [ ]:
img3 = Image.open(r"/content/Image67.jpg")

y = np.array(img3.resize((128,128)))

y = y.reshape(1, 128, 128, 3)

result2 = model.predict([y])

ind = 0
classification2 = 0

import numpy as np

for result in result2:
    result = result.tolist()
    print(result)
    
    if result[0] > result[1]:
        classification = 1
        ind = 0
    else:
        classification = 0
        ind = 1

#classification2 = np.where(result2 == np.amax(result2))[1][0]

print(str(result2[0][ind]*100) + '% Confidence This Is ' + tumor(classification2))

plt.imshow(img3)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn import svm
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [ ]:
import pandas as pd
data=pd.read_csv('Brain Tumor (1).csv')

In [ ]:
data.plot()

In [ ]:
len(data)

In [ ]:
del data['Image']

In [ ]:
#Normalizing the data

data_n = data.copy()
data_n = (data - data.min())/(data.max() - data.min())

print(data_n)

In [ ]:
dimensions=12
data_cn=pd.concat([data_n.shift(i) for i in range(0+dimensions+1)],axis=1)
print(data_cn)

In [ ]:
# Splitting the data into train, validation and test
from sklearn.model_selection import train_test_split
x=data_cn.iloc[12:,1:]
y=data_cn.iloc[12:,0]

x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.4,shuffle=False)

x_val,x_test,y_val,y_test=train_test_split(x_val,y_val,test_size=0.5,shuffle=False)

print(len(y_val))
print(len(y_test))
print(len(y_train))

In [ ]:
#PSO 
from sklearn.model_selection import train_test_split
def pso(n_particles, iterations, dimensions, inertia):

    # Range of SVR's hyperparameters (Particles' search space)
    # C, Epsilon and Gamma
    max_c = 1e4
    min_c = 1e-3
    max_e = 1e-1
    min_e = 1e-8
    max_g = 1e3
    min_g = 1e-3
    
    # Initializing particles' positions randomly, inside
    # the search space
    x = np.random.rand(n_particles, 1)*(max_c - min_c) + min_c
    y = np.random.rand(n_particles, 1)*(max_e - min_e) + min_e
    z = np.random.rand(n_particles, 1)*(max_g - min_g) + min_g

    c = np.concatenate((x,y,z), axis=1)

    # Initializing particles' parameters
    v = np.zeros((n_particles, dimensions))
    c1 = 2
    c2 = 2
    p_best = np.zeros((n_particles, dimensions))
    p_best_val = np.zeros(n_particles) + sys.maxsize  
    g_best = np.zeros(dimensions)
    g_best_val = sys.maxsize

    best_iter = np.zeros(iterations)


    # Initializing regression variables
    p_best_RGS = np.empty((n_particles), dtype = object);
    g_best_RGS = sys.maxsize

    

    # Displaying tridimensional search space
    plot(c)

    from sklearn.metrics import mean_squared_error
    from sklearn import metrics
    
    for i in range(iterations):

        for j in range(n_particles):
          # Starting Regression
          rgs = svm.SVR(C = c[j][0], epsilon = c[j][1], gamma = c[j][2])
          x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.4,shuffle=False)
          x_val,x_test,y_val,y_test=train_test_split(x_val,y_val,test_size=0.5,shuffle=False)
          # Fitting the curve
          rgs.fit(x_train,y_train)
          y_predict = rgs.predict(x_val)

          # Using Mean Squared Error to verify prediction accuracy
          mse = mean_squared_error(y_val, y_predict) 

          # If mse value for that search point, for that particle,
          # is less than its personal best point,
          # replace personal best
          if(mse < p_best_val[j]):   # mse < p_best_val[j]
          # The value below represents the current least Mean Squared Error
              p_best_val[j] = mse
              
              p_best_RGS[j] = rgs
                           

              # The value below represents the current search coordinates for
              # the particle's current least Mean Squared Error found
              p_best[j] = c[j].copy()
              
          # Using auxiliar variable to get the index of the
          # particle that found the configuration with the 
          # minimum MSE value
          aux = np.argmin(p_best_val)        
        
          if(p_best_val[aux] < g_best_val):
              # Assigning Particle's current best MSE to the Group's best    
              g_best_val = p_best_val[aux]

              # Assigning Particle's current best configuration to the Group's best
              g_best = p_best[aux].copy()

              # Group best regressor:
              # the combination of C, Epsilon and Gamma
              # that computes the best fitting curve
              g_best_RGS = p_best_RGS[aux]

        
          rand1 = np.random.random()
          rand2 = np.random.random()

          # The variable below influences directly the particle's velocity.
          # It can either make it smaller or bigger. 
          w = inertia

          # The equation below represents Particle's velocity, which is
          # the rate of change in its position
          v[j] = w*v[j] + c1*(p_best[j] - c[j])*rand1 + c2*(g_best - c[j])*rand2

          # Change in the Particle's position 
          c[j] = c[j] + v[j]

          # Below is a series of conditions that stop the particles from
          # leaving the search space
          if(c[j][2] < min_g):
            c[j][2] = min_g
          if(c[j][2] > max_g):
            c[j][2] = max_g
          if(c[j][1] < min_e):
            c[j][1] = min_e
          if(c[j][1] > max_e):
            c[j][1] = max_e
          if(c[j][0] < min_c):
            c[j][0] = min_c
          if(c[j][0] > max_c):
            c[j][0] = max_c
            
     
        # The variable below represents the least Mean Squared Error
        # of the current iteration
        best_iter[i] = g_best_val
                
        print('Best value iteration # %d = %f\n'%(i, g_best_val))

    # Coordinates found after all the iterations
    print('Group Best configuration found: ')
    print(g_best)
    print('\n')
    print('Best Regressor:\n')
    print(g_best_RGS)
    print('\n')
    # Displaying the MSE value variation throughout the iterations
    t = range(iterations)
    plt.plot(t, best_iter, label='Fitness Value')
    plt.legend()
    plt.show()

    # Displaying Particles' final configuration
    plot(c)

    # Making the prediction with the best configuration of C, Epsilon and
    # Gamma found by the particles
    x_val,x_test,y_val,y_test=train_test_split(x_val,y_val,test_size=0.5,shuffle=False)

    predict_test = g_best_RGS.predict(x_test) 
    # Displaying actual values and predicted values for
    # Group's best configuration found overall
    print(color.BOLD + 'Predictions with the Population Best Value found:\n' + color.END)
    evaluate(predict_test)  

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
# Function that displays tridimensional plot
def plot(some_list):
 
  ax = Axes3D(plt.figure())
  ax.scatter3D(some_list[:,0], some_list[:,1], some_list[:,2], color = 'r')
  ax.set_xlabel('$C$', fontsize = 20)
  ax.set_ylabel('$\epsilon$', fontsize = 25)
  ax.zaxis.set_rotate_label(False) 
  ax.set_zlabel('$\gamma$', fontsize=30, rotation = 0)
  ax.zaxis._axinfo['label']['space_factor'] = 1.0
  plt.show()

  print('\n')
  print('\n')

In [ ]:
def evaluate(predictions):

    from sklearn.metrics import mean_squared_error
    import statistics as st

    predict_test = predictions

    # To un-normalize the data:
    # Multiply the values by
    # data.to_numpy().max()

    plt.plot(range(len(y_test)), y_test, label='Real')
    plt.plot(range(len(predict_test)), predict_test, label='Predicted')
    plt.legend()
    plt.show()
    
    mse = mean_squared_error(y_test, predict_test)
    print('\n')
    print('\n')
    print('Mean Squared Error for the Test Set:\t %f' %mse)
    print('\n')
    print('\n')
    print('Predictions Average:\t %f' %((predict_test.sum()/len(predict_test))))
    print('\n')
    print('\n')
    print('Predictions Median:\t %f' %(st.median(predict_test)))
    print('\n')

In [ ]:
#Parameter of PSO
pso(50, 100, 3, 1)